In [128]:

import pandas as pd
from pandas import DataFrame
import argparse
import quandl
import pandas_datareader as web
from time import sleep
import datetime as dt
import sys
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.sectorperformance import SectorPerformances
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import os
import pickle

RAW_DATA_DIR = 'raw_data'
def load_dataset_from_disk(save_dir):
    with open(os.path.join(save_dir, "BBT"), 'rb') as f:
        transformers = pickle.load(f)
    return transformers

carl=load_dataset_from_disk(RAW_DATA_DIR)
carl.keys() #(['data', 'metadata'])
carl["data"]
# carl["metadata"][]
carl["metadata"].keys()
carl["metadata"]
carl

{'data':             1. open  2. high   3. low  4. close   5. volume
 date                                                       
 1998-01-02    63.56   63.620  62.5600     63.12    202600.0
 1998-01-05    63.00   63.190  62.5600     62.94    258100.0
 1998-01-06    62.69   62.750  62.0000     62.25    302800.0
 1998-01-07    62.00   62.000  60.5000     61.38    313300.0
 1998-01-08    61.38   61.380  59.8100     60.38    253500.0
 1998-01-09    60.00   60.380  59.4400     59.56    491500.0
 1998-01-12    58.00   59.190  58.0000     59.00    400200.0
 1998-01-13    59.00   60.000  58.9400     59.81    182600.0
 1998-01-14    59.94   60.190  59.5000     60.19    158100.0
 1998-01-15    60.00   60.310  59.5000     59.50    155200.0
 1998-01-16    59.50   59.810  59.4400     59.56    247300.0
 1998-01-20    59.56   60.000  59.5600     60.00    223800.0
 1998-01-21    60.00   60.000  58.7500     59.19    156700.0
 1998-01-22    59.00   59.000  58.1200     58.44    200700.0
 1998-01-23    5

In [129]:
from alpha_vantage.timeseries import TimeSeries
import csv
import pickle
import numpy as np
import os
import ta
import time





In [131]:
from alpha_vantage.timeseries import TimeSeries
import csv
import pickle
import numpy as np
import os
import ta
import time


def MA5(df):
    return ta.MA(df, 5)
def MA10(df):
    return ta.MA(df, 10)
def EMA20(df):
    return ta.EMA(df, 20)
def ROC(df):
    return ta.ROC(df, 5)
def ATR(df):
    return ta.ATR(df, 10)
def BBANDS(df):
    return ta.BBANDS(df, 20)
def ADX(df):
    return ta.ADX(df, 10, 5)
def MACD(df):
    return ta.MACD(df, 5, 20)
def RSI(df):
    return ta.RSI(df, 10)
def MFI(df):
    return ta.MFI(df, 10)
def CCI(df):
    return ta.CCI(df, 10)
def KELCH(df):
    return ta.KELCH(df, 20)


indicators = [MA5, MA10, EMA20, ROC, ATR, BBANDS, ta.PPSR, ta.STOK,
              ADX, MACD, RSI, MFI, CCI, KELCH]

NB_FEATURES = 30

API_KEY = 'TZZ9RPNH9YGP8SXX'
# Your premium API key for 30 API calls per minute is: TZZ9RPNH9YGP8SXX

SYM_FILE = 'S&p500companies.csv'
RAW_DATA_DIR = 'raw_data'
TRAIN_DATA_DIR = 'data'
WINDOW_SIZE = 60

TS = TimeSeries(key=API_KEY, output_format='pandas')

def get_syms():
    syms = []
    with open(SYM_FILE) as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            syms.append(row[0])
    return syms


def load_sym(sym):
    with open(os.path.join(RAW_DATA_DIR, sym),"rb") as f:
        return pickle.load(f)


def split(inputs, window=WINDOW_SIZE):
    n = len(inputs)
    indices = np.arange(window, n, window)
    split_inputs = np.split(inputs[:indices[-2]], indices[:-2])
    labels = inputs['Low'].ix[indices[1:]].as_matrix()\
        > inputs['High'].ix[indices[:-1]].as_matrix()
    assert len(split_inputs) == len(labels)
    return split_inputs, labels


def preprocess(data):
    data = data.rename(index=str, columns={x: x[3:].capitalize() for x in
                                           data.columns})
    data['Volume'] = np.log(1 + data['Volume'])
    data.index = range(len(data.index))
    for indicator in indicators:
        data = indicator(data)
    data = data.shift(-WINDOW_SIZE)[:-WINDOW_SIZE]
    return data


def split_all_data_and_save(window=WINDOW_SIZE, nb_features=NB_FEATURES):
    all_inputs = []
    all_labels = []
    for sym in os.listdir(RAW_DATA_DIR)[0:503]:
        if sym[0] != '.':
            if 'data' in data:
                inputs = preprocess(data['data'])
                if inputs.shape[0] > 2 * window:
                    print('Loading data from symbol %s...' % sym)
                    inputs, labels = split(inputs, window=window)
                    all_inputs.extend(inputs)
                    all_labels.append(labels)
    all_inputs = np.concatenate(all_inputs).reshape(-1, window, nb_features)
    all_labels = np.concatenate(all_labels).reshape(-1, 1)
    print(all_inputs.shape)
    print(all_labels.shape)
    assert all_inputs.shape[0] == all_labels.shape[0]
    with open(os.path.join(TRAIN_DATA_DIR, 'all_inputs.pkl'), 'wb') as f:
        pickle.dump(all_inputs, f)
    with open(os.path.join(TRAIN_DATA_DIR, 'all_labels.pkl'), 'wb') as f:
        pickle.dump(all_labels, f)
    return all_inputs, all_labels


def split_train_dev_test_to_file(inputs=None, labels=None):
    if inputs is None:
        with open(os.path.join(TRAIN_DATA_DIR, 'all_inputs.pkl'), 'rb') as f:
            inputs = pickle.load(f)
    if labels is None:
        with open(os.path.join(TRAIN_DATA_DIR, 'all_labels.pkl'), 'rb') as f:
            labels = pickle.load(f)
    n = inputs.shape[0]
    print(inputs.shape)
    print(labels.shape)
    indices = np.random.permutation(n)
    train_idx, eval_idx, test_idx = \
        indices[:int(0.9*n)], indices[int(0.9*n):int(0.95*n)], \
        indices[int(0.95*n):]
    pickle.dump(inputs[train_idx, :],
                open(os.path.join(TRAIN_DATA_DIR, 'train_inputs.pkl'), 'wb'))
    pickle.dump(inputs[eval_idx, :],
                open(os.path.join(TRAIN_DATA_DIR, 'eval_inputs.pkl'), 'wb'))
    pickle.dump(inputs[test_idx, :],
                open(os.path.join(TRAIN_DATA_DIR, 'test_inputs.pkl'), 'wb'))
    pickle.dump(labels[train_idx, :],
                open(os.path.join(TRAIN_DATA_DIR, 'train_labels.pkl'), 'wb'))
    pickle.dump(labels[eval_idx, :],
                open(os.path.join(TRAIN_DATA_DIR, 'eval_labels.pkl'), 'wb'))
    pickle.dump(labels[test_idx, :],
                open(os.path.join(TRAIN_DATA_DIR, 'test_labels.pkl'), 'wb'))


def get_sym_and_save(sym):
    if os.path.exists(os.path.join(RAW_DATA_DIR, sym)):
        print('Symbol %s already downloaded. Skipping...' % sym)
        return
    try:
        data, metadata = TS.get_daily(symbol=sym, outputsize='full')
    except ValueError:
        print('Symbol %s does not exist! Skipping...' % sym)
        with open(os.path.join(RAW_DATA_DIR, sym), 'w') as f:
            pickle.dump({}, f)
        return
    with open(os.path.join(RAW_DATA_DIR, sym), 'w') as f:
        pickle.dump({
            'data': data,
            'metadata': metadata
        }, f)


def get_all_raw_data(interval='1min'):
    syms = get_syms()
    for i, sym in enumerate(syms):
        print('(%d/%d) Getting data for %s...' % (i, len(syms), sym))
        get_sym_and_save(sym)


def main():
    if not os.path.exists(os.path.join(TRAIN_DATA_DIR, 'all_inputs.pkl')) \
            or not os.path.exists(os.path.join(TRAIN_DATA_DIR,
                                               'all_labels.pkl')):
        get_all_raw_data()
        split_all_data_and_save()
    split_train_dev_test_to_file()


if __name__ == '__main__':
    main()



EOFError: Ran out of input

In [123]:
# split_train_dev_test_to_file()

(43258, 60, 30)
(43258, 1)


In [124]:
#

In [125]:
#

In [ ]:
# #OLD


In [ ]:
#done


In [ ]:
# preprocess(symbols)